In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [24]:
import yfinance as yf


In [34]:
defdata = yf.download('GROW.AX TACT.XA DHOF.AX IHCB.AX VDCO.AX MVA.AX GOLD.AX')
defdata = defdata.dropna()
defdata.head(5)

[*********************100%***********************]  7 of 7 completed


Adj Close                                                    \
             DHOF.AX     GOLD.AX GROW.AX IHCB.AX     MVA.AX    TACT.XA   
Date                                                                     
2022-03-14       9.6  253.830002    3.66  102.07  23.719999  48.599998   

                        Close                      ...       Open             \
              VDCO.AX DHOF.AX     GOLD.AX GROW.AX  ...     MVA.AX    TACT.XA   
Date                                               ...                         
2022-03-14  51.220001     9.6  253.830002    3.66  ...  23.450001  48.619999   

                     Volume                                                    
           VDCO.AX  DHOF.AX GOLD.AX  GROW.AX IHCB.AX   MVA.AX TACT.XA VDCO.AX  
Date                                                                           
2022-03-14    51.5  56063.0   74148  56102.0  5706.0  29575.0  2130.0  6181.0  

[1 rows x 42 columns]

In [30]:
sharedata = yf.download('VHY.AX EX20.AX 360.AX ANZ.AX SQ2.AX CBA.AX PXA.AX ALD.AX AMC.AX BHP.AX APA.AX BXB.AX CHC.AX COL.AX WES.AX CSL.AX RMD.AX TLS.AX QAN.AX VCX.AX ANN.AX')
sharedata = sharedata.dropna()
sharedata.tail(10)

[*********************100%***********************]  21 of 21 completed


Adj Close                                                 \
              360.AX     ALD.AX AMC.AX     ANN.AX     ANZ.AX APA.AX   
Date                                                                  
2022-03-01      5.27  29.381248  15.87  25.799999  26.180000   9.93   
2022-03-02      5.05  30.239597  15.57  25.410000  25.610001   9.92   
2022-03-03      5.00  30.210001  15.42  25.540001  25.610001   9.92   
2022-03-04      4.91  29.990000  15.35  25.160000  25.299999   9.99   
2022-03-07      4.60  29.500000  14.74  25.020000  25.000000   9.89   
2022-03-08      4.40  28.270000  14.72  25.049999  24.820000   9.93   
2022-03-09      4.73  28.350000  15.01  24.990000  25.090000   9.86   
2022-03-10      5.05  27.900000  15.00  24.910000  25.770000   9.96   
2022-03-11      4.79  28.129999  14.76  24.490000  25.850000   9.95   
2022-03-14      5.00  28.870001  15.00  24.959999  26.580000  10.00   

                                                         ...     Volume  \
               BHP.AX     BXB.AX      CBA.AX     CHC.AX  ...     CSL.AX   
Date                                                     ...              
2022-03-01  46.549999   9.916658   94.820000  16.570000  ...   951047.0   
2022-03-02  48.320000  10.054527   94.699997  16.219999  ...   950661.0   
2022-03-03  50.060001   9.965898   94.510002  15.940000  ...  1220449.0   
2022-03-04  49.939999  10.005288   94.599998  15.700000  ...   996482.0   
2022-03-07  50.380001   9.828029   94.639999  15.210000  ...   874716.0   
2022-03-08  48.529999   9.739401   95.589996  15.060000  ...   911704.0   
2022-03-09  48.490002   9.670000   97.410004  15.500000  ...   663019.0   
2022-03-10  47.750000   9.860000   99.739998  15.950000  ...   934165.0   
2022-03-11  47.689999   9.720000   99.379997  15.260000  ...  1148644.0   
2022-03-14  47.360001   9.820000  102.000000  15.550000  ...   604371.0   

                                                                            \
            EX20.AX    PXA.AX      QAN.AX     RMD.AX    SQ2.AX      TLS.AX   
Date                                                                         
2022-03-01  13338.0  328584.0   8659347.0   660809.0  510528.0  29933637.0   
2022-03-02   5651.0  165429.0   9768784.0  1650526.0  647573.0  33855792.0   
2022-03-03  14114.0  260237.0   5635650.0   842201.0  437557.0  33240577.0   
2022-03-04  14621.0  165683.0  17507785.0   919249.0  424468.0  33587188.0   
2022-03-07   5453.0  727404.0  30365932.0  2198086.0  462719.0  33721649.0   
2022-03-08  10242.0  270510.0  17060109.0  1323186.0  474611.0  34104820.0   
2022-03-09  10465.0  471876.0  11087588.0  1181191.0  335186.0  38770188.0   
2022-03-10  32476.0  343138.0  20346219.0   807420.0  387851.0  34157220.0   
2022-03-11  15501.0  217438.0   7339069.0  1061190.0  363951.0  34957626.0   
2022-03-14  44074.0  113772.0   7676971.0   336951.0  242532.0  21181910.0   

                                            
                VCX.AX   VHY.AX     WES.AX  
Date                                        
2022-03-01  13667603.0  49872.0  1785880.0  
2022-03-02  10091527.0  46611.0  2466450.0  
2022-03-03  11176833.0  26484.0  3168262.0  
2022-03-04  11907971.0  47330.0  2212665.0  
2022-03-07  11129694.0  35346.0  1997817.0  
2022-03-08   9617933.0  32939.0  2273788.0  
2022-03-09   7239649.0  22960.0  1798684.0  
2022-03-10  11202564.0  47717.0  2507080.0  
2022-03-11  16492153.0  27136.0  3100139.0  
2022-03-14   6771495.0  33370.0  1710308.0  

[10 rows x 126 columns]

In [29]:
growdata = yf.download('QUAL.AX MCSG.XA FEMX.AX MCSI.XA ACDC.AX CLDD.AX')
growdata = growdata.dropna()
growdata.tail()

[*********************100%***********************]  6 of 6 completed


Adj Close                                                 Close  \
              ACDC.AX CLDD.AX FEMX.AX MCSG.XA MCSI.XA    QUAL.AX    ACDC.AX   
Date                                                                          
2022-03-14  80.849998   10.75     5.9     3.6   1.665  37.200001  80.849998   

                                    ...    Open                             \
           CLDD.AX FEMX.AX MCSG.XA  ... FEMX.AX MCSG.XA MCSI.XA    QUAL.AX   
Date                                ...                                      
2022-03-14   10.75     5.9     3.6  ...    5.93     3.6   1.665  37.099998   

             Volume                                              
            ACDC.AX  CLDD.AX  FEMX.AX  MCSG.XA  MCSI.XA QUAL.AX  
Date                                                             
2022-03-14  14974.0  25253.0  57817.0  20387.0  91645.0   68407  

[1 rows x 36 columns]

In [28]:
growdata.head(20)


Adj Close                                              Close  \
             ACDC.AX CLDD.AX FEMX.AX MCSG.XA MCSI.XA    QUAL.AX ACDC.AX   
Date                                                                      
2014-10-29       NaN     NaN     NaN     NaN     NaN  14.916200     NaN   
2014-10-30       NaN     NaN     NaN     NaN     NaN  15.146500     NaN   
2014-10-31       NaN     NaN     NaN     NaN     NaN  15.349800     NaN   
2014-11-03       NaN     NaN     NaN     NaN     NaN  15.447840     NaN   
2014-11-04       NaN     NaN     NaN     NaN     NaN  15.383900     NaN   
2014-11-05       NaN     NaN     NaN     NaN     NaN  15.687000     NaN   
2014-11-06       NaN     NaN     NaN     NaN     NaN  15.744400     NaN   
2014-11-07       NaN     NaN     NaN     NaN     NaN  15.684900     NaN   
2014-11-10       NaN     NaN     NaN     NaN     NaN  15.713800     NaN   
2014-11-11       NaN     NaN     NaN     NaN     NaN  15.720200     NaN   
2014-11-12       NaN     NaN     NaN     NaN     NaN  15.584400     NaN   
2014-11-13       NaN     NaN     NaN     NaN     NaN  15.690000     NaN   
2014-11-14       NaN     NaN     NaN     NaN     NaN  15.640000     NaN   
2014-11-17       NaN     NaN     NaN     NaN     NaN  15.480000     NaN   
2014-11-18       NaN     NaN     NaN     NaN     NaN  15.640000     NaN   
2014-11-19       NaN     NaN     NaN     NaN     NaN  15.800000     NaN   
2014-11-20       NaN     NaN     NaN     NaN     NaN  15.930000     NaN   
2014-11-21       NaN     NaN     NaN     NaN     NaN  15.910000     NaN   
2014-11-24       NaN     NaN     NaN     NaN     NaN  15.900000     NaN   
2014-11-25       NaN     NaN     NaN     NaN     NaN  16.110001     NaN   

                                    ...    Open                            \
           CLDD.AX FEMX.AX MCSG.XA  ... FEMX.AX MCSG.XA MCSI.XA   QUAL.AX   
Date                                ...                                     
2014-10-29     NaN     NaN     NaN  ...     NaN     NaN     NaN  14.91620   
2014-10-30     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.14650   
2014-10-31     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.34980   
2014-11-03     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.44784   
2014-11-04     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.38390   
2014-11-05     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.68700   
2014-11-06     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.74440   
2014-11-07     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.68490   
2014-11-10     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.71380   
2014-11-11     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.72020   
2014-11-12     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.58440   
2014-11-13     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.69000   
2014-11-14     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.67000   
2014-11-17     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.59000   
2014-11-18     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.64000   
2014-11-19     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.72000   
2014-11-20     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.94000   
2014-11-21     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.93000   
2014-11-24     NaN     NaN     NaN  ...     NaN     NaN     NaN  15.91000   
2014-11-25     NaN     NaN     NaN  ...     NaN     NaN     NaN  16.10000   

            Volume                                          
           ACDC.AX CLDD.AX FEMX.AX MCSG.XA MCSI.XA QUAL.AX  
Date                                                        
2014-10-29     NaN     NaN     NaN     NaN     NaN       0  
2014-10-30     NaN     NaN     NaN     NaN     NaN       0  
2014-10-31     NaN     NaN     NaN     NaN     NaN       0  
2014-11-03     NaN     NaN     NaN     NaN     NaN       0  
2014-11-04     NaN     NaN     NaN     NaN     NaN       0  
2014-11-05     NaN     Na